In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_excel('EC Verisi.xlsx')

In [ ]:
df1.columns = ['name', 'gender']

In [ ]:
df1['name'] = df1['name'].astype(str)

In [ ]:
df1['name'] = df1['name'].apply(lambda x: x.strip().capitalize())

In [ ]:
df1.tail(10)

,name,gender
186353,Mücahit,M
186354,Tayfun,M
186355,Emre,M
186356,Sabri,M
186357,Melik,M
186358,Emir,M
186359,Okan,M
186360,Akif,M
186361,Mert,M
186362,Faruk,M


In [ ]:
df1['gender'].value_counts()

M    129324
F     54920
       2119
Name: gender, dtype: int64

In [ ]:
df1['gender'] = df1['gender'].apply(lambda x: None if x == ' ' else x)

In [ ]:
df1 = df1.dropna(subset='gender')

In [ ]:
df1['gender'].value_counts()

M    129324
F     54920
Name: gender, dtype: int64

In [ ]:
df1['gender'].isnull().sum()

0

In [ ]:
df2 = pd.read_excel('isimler_sozluk_tr_en.xlsx', index_col=0)

In [ ]:
df2.columns = ['name', 'gender', 'origin']

In [ ]:
df2 = df2[['name', 'gender']]

In [ ]:
df2['name'] = df2['name'].astype(str)

In [ ]:
df2['name'] = df2['name'].apply(lambda x: x.strip().capitalize())

In [ ]:
df2['gender'].value_counts()

M    12193
F    11747
Name: gender, dtype: int64

In [ ]:
df2.sample(10)

,name,gender
21974,Korinne,F
9385,Çetine,F
7277,Merziye,F
14386,Breana,F
23288,Catie,F
4968,Uluçkan,M
20444,Seneca,F
3723,Özaktuğ,M
7185,Kökel,M
19719,Hildegard,F


In [ ]:
# Create a new data frame from df1, counting the occurrences of each name and gender
df1_agg = df1.groupby(['name', 'gender']).size().reset_index(name='count')
df1_agg.head()

,name,gender,count
0,A,F,2
1,A,M,3
2,Aamir,M,2
3,Aaron,M,6
4,Aashiq,M,1


In [ ]:
df1_agg['gender'].value_counts()

M    8738
F    6336
Name: gender, dtype: int64

In [ ]:
df1[df1['name']=='Piyaporn']

,name,gender
29950,Piyaporn,M
141852,Piyaporn,F


In [ ]:
# Create another data frame, counting the total occurrences of each name
df1_total = df1['name'].value_counts().reset_index()
df1_total.columns = ['name', 'total']

In [ ]:
df1_total.head()

,name,total
0,Mehmet,3522
1,Mustafa,2849
2,Ali,2554
3,Murat,2535
4,Ahmet,2451


In [ ]:
# Merge the two data frames
df1_agg = pd.merge(df1_agg, df1_total, on='name')

In [ ]:
# Calculate the probability
df1_agg['probability'] = df1_agg['count'] / df1_agg['total']

# If the probability is greater than 0.7, assign 1, else keep the probability
df1_agg['gender_prob'] = df1_agg['probability'].apply(lambda x: 1 if x > 0.7 else x)

In [ ]:
df1_agg.sample(5)

,name,gender,count,total,probability,gender_prob
14611,Çigdem,F,1,1,1.000000,1.0
13293,Vadym,M,1,1,1.000000,1.0
7803,Mphangiseni,M,1,1,1.000000,1.0
14167,Yudi,M,1,1,1.000000,1.0
11361,Seyit,M,73,75,0.973333,1.0


In [ ]:
df1_agg[df1_agg['name']=='Cabbar']

,name,gender,count,total,probability,gender_prob
2020,Cabbar,M,1,1,1.0,1.0


In [ ]:
df1_agg.to_excel('df_agg.xlsx')

In [ ]:
# Drop unnecessary columns
df1_agg.drop(['count', 'total', 'probability'], axis=1, inplace=True)

In [ ]:
df1_agg.head()

,name,gender,gender_prob
0,A,F,0.4
1,A,M,0.6
2,Aamir,M,1.0
3,Aaron,M,1.0
4,Aashiq,M,1.0


In [ ]:
# If a name does not exist in df1_agg, get the gender from df2
df_final = pd.merge(df1_agg, df2, on='name', how='outer')

In [ ]:
df_final.head()

,name,gender_x,gender_prob,gender_y
0,A,F,0.4,NaN
1,A,M,0.6,NaN
2,Aamir,M,1.0,M
3,Aaron,M,1.0,M
4,Aaron,M,1.0,F


In [ ]:
df_sorted = df_final.sort_values(by=['name', 'gender_prob'], ascending=[True, False])
df_no_duplicates = df_sorted.drop_duplicates(subset='name', keep='first')

In [ ]:
df_no_duplicates['gender_x'].fillna(df_no_duplicates['gender_y'], inplace=True)

<ipython-input-36-fde2cb93febb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['gender_x'].fillna(df_no_duplicates['gender_y'], inplace=True)


In [ ]:
df_no_duplicates['gender_prob'].fillna(1, inplace=True)

<ipython-input-37-aa4641a97538>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['gender_prob'].fillna(1, inplace=True)


In [ ]:
df_no_duplicates.reset_index().to_excel('name_gender_database.xlsx')